# Timmy's couse scheduling

## Section 1

### import

In [28]:
from gurobipy import Model, GRB, quicksum, LinExpr
import pandas as pd
import numpy as np
import re
import gurobipy as gp
import math

### data reading and preprocessing

In [29]:
# Normalize dashes in the "Fixed slot (k,t)" column
def normalize_slot(val):
    if isinstance(val, str):
        val = re.sub(r"[–—]", "-", val)      # Replace all types of dashes with '-'
        val = re.sub(r"[Ss]hifts", "Shift", val)  # Replace "shifts" or "Shifts" with "shift"
        val = re.sub(r" ", "", val)  # Replace "shifts" or "Shifts" with "shift"
    return val

def parse_fixed_slot(value):
    if pd.isna(value) or not isinstance(value, str):
        return value

    # Normalize unusual spaces (non-breaking, narrow no-break, etc.) to regular space
    value = re.sub(r'[\u00A0\u202F\u2009\u200A\u200B]', ' ', value)
    value = value.replace('\xa0', ' ')  # Also catch more hidden non-breaking spaces
    value = value.replace('–', '-')     # Replace en dash with hyphen

    # Remove extra spaces and unify format
    value = re.sub(r'\s+', '', value)

    # Updated pattern: now it can catch things like Day45Shift10 or Day45-47Shift10-12
    pattern = r"Day(\d+(?:-\d+)?)Shift(\d+(?:-\d+)?)"
    matches = re.findall(pattern, value)

    result = []

    for day_range, shift_range in matches:
        # Handle day range
        if '-' in day_range:
            start_day, end_day = map(int, day_range.split('-'))
            days = range(start_day, end_day + 1)
        else:
            days = [int(day_range)]

        # Handle shift range
        if '-' in shift_range:
            start_shift, end_shift = map(int, shift_range.split('-'))
            shifts = range(start_shift, end_shift + 1)
        else:
            shifts = [int(shift_range)]

        # Combine
        result.extend((d, s) for d in days for s in shifts)

    return result if result else value

def clean_hours(val):
    if pd.isna(val):
        return val
    if isinstance(val, str):
        # Replace en dash and em dash with regular hyphen
        val = val.replace('–', '-').replace('—', '-')

        # Remove parentheses and their contents
        val = re.sub(r"\(.*?\)", "", val).strip()

        # Handle patterns like "1-2", "1 - 2", "1.5-2.5"
        if re.match(r"^\d+(\.\d+)?\s*-\s*\d+(\.\d+)?$", val):
            parts = re.split(r"\s*-\s*", val)
            try:
                numbers = [float(p) for p in parts]
                return sum(numbers) / 2
            except ValueError:
                return val  # fallback if something goes wrong
    try:
        return float(val)
    except (ValueError, TypeError):
        return val

def expand_weekly_tasks(df, weekday_info):
    # check release
    new_rows = []   
    to_drop = []    
    for idx, row in df.iterrows():
        release = row["Release r"]
        if isinstance(release, str) and release.startswith("Weekly"):
            match = re.match(r"Weekly\s+(Mon|Tue|Wed|Thu|Fri|Sat|Sun)", release)
            if match:
                day_str = match.group(1)
                if day_str in weekday_info:
                    start_day = weekday_info[day_str]["start_day"]
                    count = weekday_info[day_str]["count"]

                    for k in range(count):
                        new_row = row.copy()
                        new_row["Task name"] = f"{row['Task name']} {k+1}"
                        new_row["Release r"] = start_day + k * 7
                        deadline = new_row["Deadline d"]
                        if isinstance(deadline, str) and deadline.startswith("Weekly"):
                            match_deadline = re.match(r"Weekly\s+(Mon|Tue|Wed|Thu|Fri|Sat|Sun)", deadline)
                            deadline = (weekday_info[match_deadline.group(1)]["start_day"] - weekday_info[day_str]["start_day"] - 1) % 7 + 1
                            new_row["Deadline d"] = new_row["Release r"] + deadline
                        new_rows.append(new_row)
                    to_drop.append(idx)
    df = df.drop(index=to_drop).reset_index(drop=True)
    if new_rows:
        df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    return df

weekday_map = {
    "Mon": {"count": 16, "start_day": 1},   # 13 Mondays, starting at day 1
    "Tue": {"count": 16, "start_day": 2},
    "Wed": {"count": 16, "start_day": 3},
    "Thu": {"count": 16, "start_day": 4},
    "Fri": {"count": 16, "start_day": 5},
    "Sat": {"count": 16, "start_day": 6},
    "Sun": {"count": 16, "start_day": 7},
}

#### data 1

In [3]:
# courses = pd.read_excel("Timmy_courses.xlsx")
# courses.replace('—', np.nan, inplace=True)
# courses["Fixed slot (k,t)"] = courses["Fixed slot (k,t)"].apply(normalize_slot)
# courses["Fixed slot (k,t)"] = courses["Fixed slot (k,t)"].apply(parse_fixed_slot)
# courses["E (hrs)"] = courses["E (hrs)"].apply(clean_hours)

# course_names = []
# course_tables = []
# start_idx = None
# for idx, row in courses.iterrows():
#     task_name = row['Task name']
#     task_type = row['Type']
#     if pd.notna(task_name) and pd.isna(task_type):
#         print(start_idx)
#         if start_idx is not None:
#             course_table = courses.iloc[start_idx+1:idx].reset_index(drop=True)
#             course_table = expand_weekly_tasks(course_table, weekday_map)
#             course_tables.append(course_table)
#         course_names.append(task_name)
#         start_idx = idx

# # Add this to capture the last course table
# if start_idx is not None:
#     course_table = courses.iloc[start_idx+1:].reset_index(drop=True)
#     course_table = expand_weekly_tasks(course_table, weekday_map)
#     course_tables.append(course_table)

# for (table, name) in zip(course_tables, course_names):
#     print(name)
#     display(table) 

#### data 2

In [30]:
courses = pd.read_excel("Timmy_courses_updated.xlsx")
courses.replace('—', np.nan, inplace=True)
courses["Fixed slot (k,t)"] = courses["Fixed slot (k,t)"].apply(normalize_slot)
courses["Fixed slot (k,t)"] = courses["Fixed slot (k,t)"].apply(parse_fixed_slot)
courses["E (hrs)"] = courses["E (hrs)"].apply(clean_hours)

course_info = []
course_tables = []
start_idx = None
for idx, row in courses.iterrows():
    task_name = row['Task name']
    task_type = row['Type']
    if pd.notna(task_name) and pd.isna(task_type):
        if start_idx is not None:
            course_table = courses.iloc[start_idx+1:idx].reset_index(drop=True)
            course_table.dropna(subset=["Task name"], inplace=True)
            course_table = expand_weekly_tasks(course_table, weekday_map)
            course_tables.append(course_table)
        course_info.append({
            "course_name": task_name, 
            "credit": row["Credit"],
            "preference": row["Preference"],
        })
        start_idx = idx

# Add this to capture the last course table
if start_idx is not None:
    course_table = courses.iloc[start_idx+1:].reset_index(drop=True)
    course_table.dropna(subset=["Task name"], inplace=True)
    course_table = expand_weekly_tasks(course_table, weekday_map)
    course_tables.append(course_table)

for (table, info) in zip(course_tables, course_info):
    display(info)
    display(table) 

{'course_name': 'IM2010 Operations Research', 'credit': 3.0, 'preference': 1.0}

,Task name,Type,Weight (%),Release r,Deadline d,"Fixed slot (k,t)",E (hrs),Notes,Credit,Preference,Minimum grade
0,HW0,Homework,0.0,1,7,NaN,2.0,NaN,NaN,NaN,NaN
1,Homework 1,Homework,5.0,22,26,NaN,4.0,NaN,NaN,NaN,NaN
2,Homework 2,Homework,5.0,29,33,NaN,4.0,NaN,NaN,NaN,NaN
3,Homework 3,Homework,5.0,78,82,NaN,4.0,NaN,NaN,NaN,NaN
4,Final Project Proposal (FPP),Homework,0.0,36,40,NaN,3.0,NaN,NaN,NaN,NaN
5,Midterm Project,Homework,20.0,64,82,NaN,12.0,NaN,NaN,NaN,NaN
6,Final Project Video (FPV),Homework,0.0,85,96,NaN,8.0,NaN,NaN,NaN,NaN
7,Final Project Report (FPR),Homework,25.0,85,96,NaN,10.0,NaN,NaN,NaN,NaN
8,Midterm Exam,Exam,12.0,NaN,NaN,"[(48, 2), (48, 3), (48, 4)]",6.0,Fixed slot,NaN,NaN,NaN
9,Final Exam,Exam,18.0,NaN,NaN,"[(106, 2), (106, 3), (106, 4)]",6.0,Fixed slot,NaN,NaN,NaN


{'course_name': 'MATH4008 Calculus III', 'credit': 2.0, 'preference': 0.8}

,Task name,Type,Weight (%),Release r,Deadline d,"Fixed slot (k,t)",E (hrs),Notes,Credit,Preference,Minimum grade
0,Worksheet 1,Homework,7.0,8,22,NaN,2.0,NaN,NaN,NaN,NaN
1,Worksheet 2,Homework,7.0,22,36,NaN,2.0,NaN,NaN,NaN,NaN
2,Worksheet 3,Homework,7.0,36,50,NaN,2.0,NaN,NaN,NaN,NaN
3,WeBWorK,Homework,10.0,1,56,NaN,4.0,Online assignments,NaN,NaN,NaN
4,Quiz 1,Exam,10.0,15,NaN,"[(18, 11)]",3.0,17:30–18:20,NaN,NaN,NaN
5,Quiz 2,Exam,10.0,36,NaN,"[(39, 11)]",3.0,17:30–18:20,NaN,NaN,NaN
6,Final Exam,Exam,50.0,NaN,NaN,"[(57, 7), (57, 8), (57, 9)]",8.0,13:30–16:30,NaN,NaN,NaN


{'course_name': 'MATH4010 Calculus IV – Applications in Economics and Business',
 'credit': 2.0,
 'preference': 0.8}

...

,Task name,Type,Weight (%),Release r,Deadline d,"Fixed slot (k,t)",E (hrs),Notes,Credit,Preference,Minimum grade
0,Class Participation W1,Homework,1.0,45,45,"[(45, 2)]",0.5,NaN,NaN,NaN,NaN
1,Class Participation W2,Homework,1.0,52,52,"[(52, 2)]",0.5,NaN,NaN,NaN,NaN
2,Class Participation W3,Homework,1.0,59,59,"[(59, 2)]",0.5,NaN,NaN,NaN,NaN
3,Class Participation W4,Homework,1.0,66,66,"[(66, 2)]",0.5,NaN,NaN,NaN,NaN
4,Class Participation W5,Homework,1.0,73,73,"[(73, 2)]",0.5,NaN,NaN,NaN,NaN
5,Class Participation W6,Homework,1.0,80,80,"[(80, 2)]",0.5,NaN,NaN,NaN,NaN
6,Class Participation W7,Homework,1.0,87,87,"[(87, 2)]",0.5,NaN,NaN,NaN,NaN
7,Class Participation W8,Homework,1.0,94,94,"[(94, 2)]",0.5,NaN,NaN,NaN,NaN
8,Class Participation W9,Homework,1.0,101,101,"[(101, 2)]",0.5,NaN,NaN,NaN,NaN
9,Class Participation W10,Homework,1.0,108,108,"[(108, 2)]",0.5,NaN,NaN,NaN,NaN


{'course_name': 'IM3004 Organizational Behavior',
 'credit': 3.0,
 'preference': 0.4}

,Task name,Type,Weight (%),Release r,Deadline d,"Fixed slot (k,t)",E (hrs),Notes,Credit,Preference,Minimum grade
0,Case Study Presentation,Homework,25.0,1,15,"[(15, 7)]",8.0,Present during Monday 14:20–17:20,NaN,NaN,NaN
1,Midterm Exam,Exam,30.0,NaN,NaN,"[(50, 7)]",6.0,Mon 4/7,NaN,NaN,NaN
2,Final Exam,Exam,30.0,NaN,NaN,"[(106, 7)]",6.0,Mon 6/2,NaN,NaN,NaN
3,Participation W1,Homework,1.0,42,42,"[(42, 7)]",0.5,NaN,NaN,NaN,NaN
4,Participation W2,Homework,1.0,47,47,"[(47, 7)]",0.5,NaN,NaN,NaN,NaN
5,Participation W3,Homework,2.0,52,52,"[(52, 7)]",1.0,NaN,NaN,NaN,NaN
6,Participation W5,Homework,1.0,62,62,"[(62, 7)]",0.5,NaN,NaN,NaN,NaN
7,Participation W6,Homework,1.0,67,67,"[(67, 7)]",0.5,NaN,NaN,NaN,NaN
8,Participation W7,Homework,1.0,72,72,"[(72, 7)]",0.5,NaN,NaN,NaN,NaN
9,Participation W8,Homework,1.0,77,77,"[(77, 7)]",0.5,NaN,NaN,NaN,NaN


{'course_name': 'MGT1002 Accounting Principles (2)',
 'credit': 3.0,
 'preference': 0.8}

,Task name,Type,Weight (%),Release r,Deadline d,"Fixed slot (k,t)",E (hrs),Notes,Credit,Preference,Minimum grade
0,Quiz,Exam,4.0,15,22,"[(22, 10)]",1.0,3/12 in‑class,NaN,NaN,NaN
1,Project,Homework,6.0,99,103,"[(103, 10)]",5.0,Report day 5/23,NaN,NaN,NaN
2,Exam 1,Exam,27.0,NaN,NaN,"[(31, 10)]",6.0,3/19 in‑class,NaN,NaN,NaN
3,Exam 2,Exam,27.0,NaN,NaN,"[(66, 10)]",6.0,4/23 in‑class,NaN,NaN,NaN
4,Exam 3,Exam,26.0,NaN,NaN,"[(109, 10)]",6.0,6/4 in‑class,NaN,NaN,NaN
5,TA Session W1,Homework,1.0,45,45,"[(45, 10)]",0.5,NaN,NaN,NaN,NaN
6,TA Session W2,Homework,1.0,52,52,"[(52, 10)]",0.5,NaN,NaN,NaN,NaN
7,TA Session W3,Homework,1.0,59,59,"[(59, 10)]",1.0,NaN,NaN,NaN,NaN
8,TA Session W5,Homework,1.0,73,73,"[(73, 10)]",0.5,NaN,NaN,NaN,NaN
9,TA Session W6,Homework,1.0,80,80,"[(80, 10)]",0.5,NaN,NaN,NaN,NaN


#### data 3

In [31]:
day_limit = pd.read_excel("Timmy_Personal_Data.xlsx", usecols=[0, 1])
display(day_limit.head())

Timmy_status = pd.read_excel("Timmy_Personal_Data.xlsx", usecols=[2, 3, 4], nrows = 1)
display(Timmy_status.head())

Timmy_efficiency = pd.read_excel("Timmy_Personal_Data.xlsx", usecols=[6, 7], nrows=3)
display(Timmy_efficiency.head())

Timmy_efficiency.rename(columns={"Unnamed: 6": "Type"}, inplace = True)
display(Timmy_efficiency.head())

# fix
new_row = pd.DataFrame([{"Type": "Online Quiz", "Base efficiency": 0.0}])
Timmy_efficiency = pd.concat([Timmy_efficiency, new_row], ignore_index=True)
Timmy_efficiency.loc[Timmy_efficiency["Type"] == "Acativity", "Type"] = "Activity"
display(Timmy_efficiency.head())

,Day,Workload Limits
0,1,6
1,2,6
2,3,6
3,4,6
4,5,6


,Penalty coefficient,Circadian peak,Full cosine swing
0,1,5,1


,Unnamed: 6,Base efficiency
0,Homework,0.6
1,Exam,1.0
2,Acativity,1.0


,Type,Base efficiency
0,Homework,0.6
1,Exam,1.0
2,Acativity,1.0


,Type,Base efficiency
0,Homework,0.6
1,Exam,1.0
2,Activity,1.0
3,Online Quiz,0.0


### parameter

In [32]:
num_course = len(course_info)
num_day = day_limit.shape[0] # 2/17-6/8
num_shift = 16
minimum_grade = 60

# Indices
I = range(num_course)                # Courses
J = {i: course_tables[i] for i in I}  # Tasks per course i
K = range(num_day)                # Days
T = range(num_shift)               # Shifts per day

# === Calculation Part ===
w = {i: course_info[i]["credit"] for i in I}           
S = {(i, idx): row["Weight (%)"] for i in I for idx, row in J[i].iterrows()}
E = {
    (i, idx): (row["E (hrs)"] + len(row["Fixed slot (k,t)"])) 
              if isinstance(row["Fixed slot (k,t)"], list) 
              else row["E (hrs)"]
    for i in I
    for idx, row in J[i].iterrows()
    if not pd.isna(row["E (hrs)"])
}
r = {(i, idx): row["Release r"] for i in I for idx, row in J[i].iterrows()}
d = {(i, idx): row["Deadline d"] for i in I for idx, row in J[i].iterrows()}
B = minimum_grade  # Minimum grade
F = []
for i in I:
    for j in range(J[i].shape[0]):
        value = course_tables[i]["Fixed slot (k,t)"][j]
        if isinstance(value, (list, tuple)) and not (pd.isna(value).all() if isinstance(value, (np.ndarray, list, tuple)) else pd.isna(value)):
            for k_t in value:
                if isinstance(k_t, (list, tuple)) and len(k_t) == 2 and not any(pd.isna(x) for x in k_t):
                    k, t = k_t
                    F.append((i, j, k, t))
F = np.array(F)

H_star = {k: day_limit["Workload Limits"][k] for k in K}  # Threshold for overload
beta = Timmy_status["Penalty coefficient"][0]  # Penalty weight

# Define P_ijkt
eta_type = {(i, idx): row["Base efficiency"] 
            for i in I 
            for idx, row in course_tables[i].merge(Timmy_efficiency, on="Type").iterrows()}
theta = {i: course_info[i]["preference"] for i in I}  # Enthusiasm per course
h_t = {t: t + 0.5 for t in T[:num_shift]}  # Midpoint hour for shift
h_peak = Timmy_status["Circadian peak"][0]  # Peak energy time
a = 1  # Circadian amplitude
P = {}  # dictionary to hold expressions
for i in I:
    enthusiasm = 1 + theta[i]# enthusiasm effect
    for j in range(J[i].shape[0]):
        eta = eta_type[i, j]# task-type effect
        for k in K:
            for t in T:
                circadian = 1 + a * math.cos((2 * math.pi / 24) * (h_t[t] - h_peak))# circadian effect
                P[i, j, k, t] = eta * enthusiasm * circadian

# === Output Part ===
# print("weight per course\n", w)
# print("weight per task\n", S)
# print("task required time\n", E)
# print("task release times\n", r)
# print("task deadlines\n", d)
# print("minimum grades each course (all the same)\n", B)
# print("max study time each day\n", H_star)
# print("penalty (if studying too long)\n", beta)
# print("Timmy's efficiency to each task\n", eta_type)
# print("Timmy's enthusiasm to each course\n", theta)
# print("midpoint hour for shift\n", h_t)
# print("peak energy time", h_peak)
print(F)

[[  0   8  48   2]
 [  0   8  48   3]
 [  0   8  48   4]
 [  0   9 106   2]
 [  0   9 106   3]
 [  0   9 106   4]
 [  0  11  44   2]
 [  0  11  44   3]
 [  0  11  44   4]
 [  0  12  51   2]
 [  0  12  51   3]
 [  0  12  51   4]
 [  0  13  58   2]
 [  0  13  58   3]
 [  0  13  58   4]
 [  0  14  65   2]
 [  0  14  65   3]
 [  0  14  65   4]
 [  0  15  72   2]
 [  0  15  72   3]
 [  0  15  72   4]
 [  1   4  18  11]
 [  1   5  39  11]
 [  1   6  57   7]
 [  1   6  57   8]
 [  1   6  57   9]
 [  2   4  82  11]
 [  2   5  96  11]
 [  2   6 111   7]
 [  2   6 111   8]
 [  2   6 111   9]
 [  3  10  51   8]
 [  3  11 106   8]
 [  4   0  26   4]
 [  4   1  33   4]
 [  4   2  40   4]
 [  4   3  54   4]
 [  4   4  68   4]
 [  4   5  75   4]
 [  4   6  56   3]
 [  4   6  56   4]
 [  4   6  56   5]
 [  5   0  45   2]
 [  5   1  52   2]
 [  5   2  59   2]
 [  5   3  66   2]
 [  5   4  73   2]
 [  5   5  80   2]
 [  5   6  87   2]
 [  5   7  94   2]
 [  5   8 101   2]
 [  5   9 108   2]
 [  5  20  4

In [46]:
# In[model]
model = Model("StudySchedule")

In [47]:
# Add decision variables y[k, t, i, j] ∈ {0,1}
y = {}
for k in K:
    for t in T:
        for i in I:
            for j in range(J[i].shape[0]):
                y[k, t, i, j] = model.addVar(vtype=GRB.BINARY, name=f"y_{k}_{t}_{i}_{j}")

model.update()

# Compute A_{i,j} = ∑_{k,t} P_{i,j,k,t} * y_{k,t,i,j}
A = {(i, j): quicksum(P[i, j, k, t] * y[k, t, i, j] for k in K for t in T)
     for i in I for j in range(J[i].shape[0])}

# Add continuous variables X_{i,j} ∈ [0, 1]
X = {(i, j): model.addVar(lb=0, ub=1, name=f"X_{i}_{j}") for i in I for j in range(J[i].shape[0])}
model.update()

# Add constraints X_{i,j} ≤ A_{i,j} / E_{i,j}
for (i, j) in X:
    model.addConstr(X[i, j] <= A[i, j] / E[i, j], name=f"XLimit_{i}_{j}")

# Compute grades G_i = ∑_{j} S_{i,j} * X_{i,j}
G = {i: quicksum(S[i, j] * X[i, j] for j in range(J[i].shape[0])) for i in I}

# Overload penalties for exceeding H*_k
overload_penalty = {}
for k in K:
    total_hours = quicksum(y[k, t, i, j] for t in T for i in I for j in range(J[i].shape[0]))
    overload = model.addVar(lb=0, name=f"Overload_{k}")
    model.addConstr(overload >= total_hours - H_star[k], name=f"OverloadConstr_{k}")
    overload_penalty[k] = overload

# Objective: Maximize ∑ w_i * G_i - β * ∑ overload penalties
model.setObjective(
    quicksum(w[i] * G[i] for i in I) - beta * quicksum(overload_penalty[k] for k in K),
    GRB.MAXIMIZE
)

# Constraint: Only one task per shift
for k in K:
    for t in T:
        model.addConstr(
            quicksum(y[k, t, i, j] for i in I for j in range(J[i].shape[0])) <= 1,
            name=f"OneTaskPerShift_{k}_{t}"
        )

# print("deadline")
# Constraint: Do not assign after deadline
for i in I:
    for j in range(J[i].shape[0]):
        deadline_day = d[i, j] #
        if np.isnan(deadline_day): 
            continue
        for k in K:
            if k > deadline_day - 1: #
                for t in T:
                    # print(f"({i}, {j}, {k}, {t})")
                    model.addConstr(y[k, t, i, j] == 0, name=f"Deadline_{k}_{t}_{i}_{j}")

# print("release")
# Constraint: Do not assign before release
for i in I:
    for j in range(J[i].shape[0]):
        release_day = r[i, j]
        if isinstance(release_day, str):
            continue
        if np.isnan(release_day): 
            continue
        for k in K:
            if k < release_day - 1: #
                for t in T:
                    # print(f"({i}, {j}, {k}, {t})")
                    model.addConstr(y[k, t, i, j] == 0, name=f"Release_{k}_{t}_{i}_{j}")

# Constraint: Breaks - no more than 4 study blocks in any 6-shift window
for k in K:
    for t0 in range(len(T) - 5):
        model.addConstr(
            quicksum(y[k, t, i, j] for t in range(t0, t0 + 6) for i in I for j in range(J[i].shape[0])) <= 4,
            name=f"Breaks_{k}_{t0}"
        )

# print("fixed slot")
# Constraint: fixed slots
for i, j, k, t in F:
    model.addConstr(y[k - 1, t, i, j] == 1, name=f"blocked_shift_{k}_{t}_{i}_{j}") #
    # print(f"({i}, {j}, {k}, {t})")

# Constraint: Minimum grade requirement
for i in I:
    model.addConstr(G[i] >= B, name=f"MinGrade_{i}")

In [50]:
# Optimize the model
model.optimize()

# Print results if feasible/optimal
if model.status == GRB.OPTIMAL:
    print("\n--- Schedule ---")
    for k in K:
        for t in T:
            for i in I:
                for j in range(J[i].shape[0]):
                    if y[k, t, i, j].X > 0.5:
                        print(f"Day {k+1}, Shift {t+1}: {course_info[i]["course_name"]}, Task {J[i]["Task name"][j]}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 217242 rows, 256014 columns and 2007099 nonzeros
Model fingerprint: 0x689b5d53
Variable types: 254 continuous, 255760 integer (255760 binary)
Coefficient statistics:
  Matrix range     [7e-03, 5e+01]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolved: 3187 rows, 31858 columns, 224394 nonzeros

Continuing optimization...


Explored 1 nodes (286 simplex iterations) in 0.08 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 2: 2210.44 1547.27 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.210441936575e+03, best bound 2.210441936575e+03, gap 0.0000%

--- Schedule ---
Day 1, Shift 1: MGT1002 Accounting Principles 

### Desciption

In [33]:
# Prepare list to collect schedule entries
schedule_entries = []

if model.status == GRB.OPTIMAL:
    print("\n--- Schedule ---")
    for k in K:
        for t in T:
            for i in I:
                for j in range(J[i].shape[0]):
                    if y[k, t, i, j].X > 0.5:
                        course_name = course_info[i]["course_name"]
                        task_name = J[i]["Task name"][j]
                        print(f"Day {k}, Shift {t}: {course_name}, Task {task_name}")

                        # Append to list for DataFrame
                        schedule_entries.append({
                            "Day": k,
                            "Shift": t,
                            "Course": course_name,
                            "Task": task_name
                        })

# Create DataFrame
df_schedule = pd.DataFrame(schedule_entries)


--- Schedule ---
Day 1, Shift 1: MATH4008 Calculus III, Task WeBWorK
Day 1, Shift 2: IM2010 Operations Research, Task Final Exam
Day 2, Shift 3: MGT1002 Accounting Principles (2), Task Exam 1
Day 4, Shift 0: MATH4008 Calculus III, Task WeBWorK
Day 4, Shift 2: IM3004 Organizational Behavior, Task Case Study Presentation
Day 6, Shift 0: CSIE1212 Data Structures and Algorithms, Task Final Exam
Day 7, Shift 0: MGT1002 Accounting Principles (2), Task Exam 3
Day 8, Shift 0: MGT1002 Accounting Principles (2), Task Exam 3
Day 8, Shift 5: IM3004 Organizational Behavior, Task Case Study Presentation
Day 9, Shift 0: MATH4008 Calculus III, Task Worksheet 1
Day 10, Shift 0: IM3004 Organizational Behavior, Task Case Study Presentation
Day 12, Shift 0: IM3004 Organizational Behavior, Task Case Study Presentation
Day 13, Shift 0: IM3004 Organizational Behavior, Task Case Study Presentation
Day 14, Shift 1: IM3004 Organizational Behavior, Task Case Study Presentation
Day 16, Shift 0: ECON1023 Principl

In [34]:
task_count = df_schedule.groupby(["Course", "Task"]).size().reset_index(name="Count")
print(task_count)

                                      Course           Task  Count
0    CSIE1212 Data Structures and Algorithms     Earth Game      1
1    CSIE1212 Data Structures and Algorithms     Final Exam      2
2    CSIE1212 Data Structures and Algorithms     Homework 0      3
3    CSIE1212 Data Structures and Algorithms     Homework 1      5
4    CSIE1212 Data Structures and Algorithms     Homework 2      5
..                                       ...            ...    ...
116        MGT1002 Accounting Principles (2)  TA Session W5      1
117        MGT1002 Accounting Principles (2)  TA Session W6      1
118        MGT1002 Accounting Principles (2)  TA Session W7      1
119        MGT1002 Accounting Principles (2)  TA Session W8      1
120        MGT1002 Accounting Principles (2)  TA Session W9      1

[121 rows x 3 columns]


In [35]:
tasks_per_day = df_schedule.groupby("Day").size().reset_index(name="Task Count")
print(tasks_per_day)

     Day  Task Count
0      1           2
1      2           1
2      4           2
3      6           1
4      7           1
..   ...         ...
97   110           4
98   111           2
99   112           3
100  113           1
101  114           3

[102 rows x 2 columns]


In [36]:
heatmap = df_schedule.pivot_table(index="Day", columns="Shift", aggfunc="size", fill_value=0)
print(heatmap)

Shift  0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
Day                                                                  
1       0   1   1   0   0   0   0   0   0   0   0   0   0   0   0   0
2       0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0
4       1   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0
6       1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
7       1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
..     ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
110     1   0   1   0   0   1   0   0   0   0   0   0   0   1   0   0
111     1   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0
112     1   0   0   1   1   0   0   0   0   0   0   0   0   0   0   0
113     1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
114     1   0   1   0   0   0   0   1   0   0   0   0   0   0   0   0

[102 rows x 16 columns]
